这是一个定制化的对股票估值的『毛估估』算法，主要是解决个人的两个需求，
1. 对不赢利的年轻公司进行估值；
2. 将不成熟的、成熟的、低增长的不同公司股票放在一起比较（虽然定量的比较一定是不太靠谱的）。

大体的做法是，
1. 在通常的DCF方法中假设的『快速成长阶段』之前，增加了一个『不稳定成长阶段』；
2. 在『不稳定成长阶段』中，通常处于亏损状态，因为需要投入大量成本来扩大生产和建立市场地位，随着销售规模的快速扩张带来营收的高速增长，同时成本不会以相同的速度增长，从而在期末实现一个合理的利润率；
3. 在『快速成长阶段』中，我们假设企业通过将绝大部分利润投入了再生产，通过提升销量、提高定价、优化成本等方法实现收益的稳定增长。因此不计入此阶段各年收益，这与传统做法有所区别。
4. 公平起见，『不稳定成长阶段』和『快速成长阶段』共享总年数

具体细节可见下文代码及注释。

In [5]:
# 环境和包配置
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import io

pd.set_option( 'display.precision',2)

In [6]:
# 工具函数
def read_csv(csv_str):
    return pd.read_csv(io.StringIO(csv_str), comment='#', index_col=0, skipinitialspace=True)

# 估值计算过程, 针对传统 DCF 方法有一些变化, 但是总体而言并不关键, 关键的还是前面所说对于"Unstable.期末EPS"的估计
def generalized_dcf(global_params, init_rps, unstable_years, unstable_growth, unstable_margin, rapid_growth):
    """
    计算推广的DCF, 支持对不赢利或赢利不稳定的"年轻"公司的估值. 计算分为两个阶段, 见代码中注释。

    Parameters
    ----------
    
    """
    total_growth_years = global_params["成长阶段总年数"]
    terminal_years = global_params["Terminal.年数"]
    terminal_growth = global_params["Terminal.增长率"]
    discount = global_params["折现率"]

    # 1. 估计 "Unstable.期末EPS", 作为快速成长阶段计算的起点
    unstable_eps = init_rps * (1+unstable_growth) ** unstable_years * unstable_margin

    # 2. 估计 "Rapid.期末EPS", 作为永续阶段计算的起点
    rapid_years = total_growth_years - unstable_years
    rapid_eps = unstable_eps * (1+rapid_growth)**rapid_years

    # 3. 计算内在价值. 
    # 与传统 DCF 方法不同, 从利润留存/分红的角度来考虑, 在这个阶段为了保证快速成长, 绝大部分收益都应投入了扩大生产,
    # 所以并不计入快速成长阶段中的各年收益. 同时, 这样一来成长型行业与低增长型在逻辑上更有可比性(虽然一般不会这么干, 纯是心理作用).
    # 总的来说, 这个调整并不关键, 但因为不计成长阶段收益, 因此最终估值会相对其他一些估值方法偏小.
    q = (1+terminal_growth)/(1+discount) # 中间变量
    fair_values = rapid_eps*q*(1-q**terminal_years) / ((1-q)*(1+discount)**total_growth_years)
    return unstable_eps, rapid_years, rapid_eps, fair_values

# 简易包装
def perform_valuation(global_params, data, current_prices):
    data = data.join(current_prices.rename("现价"))

    # 计算估值
    init_rps = data["期初RPS"]
    unstable_years = data["Unstable.年数"]
    unstable_growth = data["Unstable.增长率"]/100
    unstable_margin = data["Unstable.期末利润率"]/100
    rapid_growth = data["Rapid.增长率"]/100
    unstable_eps, rapid_years, rapid_eps, fair_values = generalized_dcf(global_params, init_rps, unstable_years, unstable_growth, unstable_margin, rapid_growth)
    output = pd.DataFrame({
        "期初RPS" : data["期初RPS"],
        "Unstable.年数" : data["Unstable.年数"],
        "Unstable.增长率" : data["Unstable.增长率"],
        "Unstable.期末利润率" : data["Unstable.期末利润率"],
        "Unstable.期末EPS" : unstable_eps,
        "Rapid.增长率" : data["Rapid.增长率"],
        "Rapid.期末EPS" : rapid_eps,
        "内在价值" : fair_values
    })

    # 代入现价, 比较安全边际
    prices = data["现价"]
    margin_of_safety = (fair_values - prices) / fair_values
    output["现价"] = prices
    output["安全边际%"] = np.array(margin_of_safety.T * 100)
    init_eps = unstable_eps / (1+rapid_growth) ** unstable_years  # 期初估测EPS, 对于成熟公司来说等同于 EPS, 对于不成熟公司来说等于Unstable.期末EPS对增长率进行折算
    output["估测PE"] = prices / np.array(init_eps) # = 现价 / 期初EPS, 对于成熟公司来说等同于 PE
    return output


In [7]:
# 全局参数
global_params_table = read_csv(
"""
,                    默认
成长阶段总年数,        10.0
Terminal.年数,     10.0
Terminal.增长率,    0.04
折现率,               0.1
"""
)

# 各公司特征数据
# ------
# 相较于传统的DCF计算, 在快速成长阶段和永续增长阶段之外, 增加了一个不稳定成长阶段, 主要是为了支持对不赢利或赢利不稳定的"年轻"公司的估值.
# 公平起见, 这个阶段与快速成长阶段共享总年数, 其期末EPS的估计需要根据个人对特定公司的理解来定, 也是这个毛估估方法的核心.
# -------
# Unstable.年数: 成熟公司设置为0; 未赢利公司考虑公司发展情况设置
# Unstable.期末EPS: 成熟公司设置为近期财报公布的扣非每股收益; 未赢利公司根据营收等预期估算, 在数据表中作相应说明
# Rapid.增长率: 成熟公司参考10年扣非每股收益增长率; 未赢利公司可参考行业内成熟企业的数值

# 港股数据来自 futu，美股数据来自 guru
data_202303 = read_csv(
"""
名称,   期初RPS,  Unstable.年数,   Unstable.增长率,  Unstable.期末利润率,   Rapid.增长率
网易,   32.89,    0,              0,               21.0,                16.6
腾讯,   64.88,    0,              0,               33.4,                20.0
快手,   24.46,    4.75,           20.0,            20.0,                16.6          # 近3年营收增长率66.5%, 近1年13.4%; 行业20%的净利率
京东,   334.96,   1.75,           30.0,            2.5,                 16.6          # 近5年营收增长率43.2%, 近3年19.4%, 近1年-1.9%; 略低于同行costco的净利率2.5%
美团,   39.76,    1.75,           30.0,            20.0,                20.0          # 近5年营收增长率58.3%, 近3年10.1%, 近1年26.6%, 线下优势强劲; 行业20%净利率
拼多多, 13.36,    1.75,           50.0,            24.0,                20.0          # 近5年营收增长率96.4%, 近3年51.8%, 近1年25.9%, 海外进展迅速; 行业20%净利率
苹果,   23.97,    0,              0,               24.6,                14.9
微软,   27.22,    0,              0,               33.1,                16.8
"""
)

data_202305 = read_csv(
"""
名称,   期初RPS,  Unstable.年数,   Unstable.增长率,  Unstable.期末利润率,   Rapid.增长率
网易,   31.11,    0,              0,               24.6,                16.6
腾讯,   65.41,    0,              0,               33.2,                16.6
快手,   25.02,    4.5,            20.0,            20.0,                16.6          # 近3年营收增长率66.5%, 近1年13.4%; 行业20%的净利率
京东,   362.1,    1.5,            20.0,            2.5,                 16.6          # 近5年营收增长率43.2%, 近3年19.4%, 近1年-1.9%; 略低于同行costco的净利率2.5%
美团,   41.26,    1.5,            30.0,            20.0,                16.6          # 近5年营收增长率58.3%, 近3年10.1%, 近1年26.6%, 线下优势强劲; 行业20%净利率
拼多多, 15.54,    1.5,            50.0,            24.0,                16.6          # 近5年营收增长率96.4%, 近3年51.8%, 近1年25.9%, 海外进展迅速; 近期净利率超过25%, 行业20%净利率
苹果,   24.02,    0,              0,               24.5,                14.9
微软,   27.75,    0,              0,               33.3,                16.8
"""
)

data_202309 = read_csv(
f"""
名称,   期初RPS,  Unstable.年数,   Unstable.增长率,  Unstable.期末利润率,   Rapid.增长率
网易,   {32.53+16.02-15.8},    0,              0,               {(6.97+5.05-3.46)/(32.53+16.02-15.8)*100},                16.6
腾讯,   {64.88+33.9-32.75},    0,              0,               {(22.12+5.95-5.15)/(64.88+33.9-32.75)*100},                16.6
快手,   {24.46+13.25-11.69},    4.25,            20.0,            20.0,                16.6          # 近3年营收增长率66.5%, 近1年13.4%; 行业20%的净利率
京东,   {368.35+181.09-186.64},    1.25,            20.0,            2.5,                 16.6          # 近5年营收增长率43.2%, 近3年19.4%, 近1年-1.9%; 略低于同行costco的净利率2.5%
美团,   {39.76+21.99-18.38},    1.25,            30.0,            20.0,                16.6          # 近5年营收增长率58.3%, 近3年10.1%, 近1年26.6%, 线下优势强劲; 行业20%净利率
拼多多, 16.15,    1.25,            50.0,            {4.05/16.15*100},                16.6          # 近5年营收增长率96.4%, 近3年51.8%, 近1年25.9%, 海外进展迅速; 近期净利率超过25%, 行业20%净利率
苹果,   24.10,    0,              0,               {5.95/24.10*100},                14.9
微软,   28.36,    0,              0,               {9.69/28.36*100},                16.8
英伟达, 13.11,    0,              0,               {4.14/13.11*100},                36             # 近一年利润增长率接近36%, 这种高速能持续 5-10 年吗, 不太确定, 估且先用来算着
"""
)

data_202312 = read_csv(
f"""
名称,   期初RPS,  Unstable.年数,   Unstable.增长率,  Unstable.期末利润率,   Rapid.增长率
网易,   {32.53+25.04-22.76},    0,              0,               {(6.97+7.74-5.54)/(32.53+25.04-22.76)*100},                16.6
腾讯,   {64.88+51.91-47.19},    0,              0,               {(22.12+10.15-9.5)/(64.88+51.91-47.19)*100},                16.6
快手,   {24.46+20.25-16.95},    4,            20.0,            20.0,                16.6          # 近3年营收增长率66.5%, 近1年13.4%; 行业20%的净利率; Q3 营收同比增长rmb 20.84%(hkd 17.46%), 经营利润率来到3.47%, 还有很大提升空间
京东,   {368.35+266.84-261.04},    1,            20.0,            2.5,                 16.6          # 近5年营收增长率43.2%, 近3年19.4%, 近1年-1.9%; 略低于同行costco的净利率2.5%, 目前已接近目标
美团,   {39.76+35.44-28.55},    1,            30.0,            20.0,                16.6          # 近5年营收增长率58.3%, 近3年10.1%, 近1年26.6%; 行业20%净利率; 线下优势受到挑战, 营收增速放缓
拼多多, 19.08,    1,            50.0,            {5.25/19.08*100},                16.6          # 近5年营收增长率96.4%, 近3年51.8%, 近1年53.6%, 海外进展迅速; 近期净利率超过25%, 行业20%净利率
苹果,   24.22,    0,              0,               {6.12/24.22*100},                16.6
微软,   29.24,    0,              0,               {10.46/29.24*100},                16.6
英伟达, 18,    0,              0,               {8.11/18*100},                30             # 近一年利润增长率接近173.2%, 3年 24%, 5年 18%, 考虑到 AI 行业的爆发，先用 30% 算吧
"""
)

# 各公司价格数据
prices_table = read_csv(
"""
名称,   20230216,   20230322,    20230530,  20230918,   20231229
网易,   138.2,      134.0,       139.5,     158.3,      140.6
腾讯,   382.6,      347.2,       316.2,     312.2,      293.6
快手,   61.9,       54,          54.5,      63.7,       52.95
京东,   212.6,      152.9,       130.0,     121.0,      112.5
美团,   148.3,      130.3,       116.4,     122.0,      81.9
拼多多, 94.46,      78.93,       71.42,     99.81,      146.31
苹果,   155.33,     159.28,      175.43,    177.97,     192.53
微软,   269.32,     273.78,      332.89,    329.06,     376.04
英伟达, 219.939,    264.627,     401.03,    439.66,     495.22
BRK,   469045,     460515.03,    486650.00, 562800,     542625.03
""")

In [8]:
# 计算打印各公司估值
global_params = global_params_table["默认"];

print("global_params:")
display(global_params.to_frame())

# print("data_202211:")
# output_202211 = perform_valuation(global_params, data_202211, current_prices)
# display(output_202211)

print("data_202303:")
output_202303 = perform_valuation(global_params, data_202303, prices_table["20230322"])
display(output_202303)

print("data_202305:")
output_202305 = perform_valuation(global_params, data_202305, prices_table["20230530"])
display(output_202305)

print("data_202309:")
output_202309 = perform_valuation(global_params, data_202309, prices_table["20230918"])
display(output_202309)

print("data_202312:")
output_202312 = perform_valuation(global_params, data_202312, prices_table["20231229"])
display(output_202312)

global_params:


,默认
成长阶段总年数,10.00
Terminal.年数,10.00
Terminal.增长率,0.04
折现率,0.10


data_202303:


,期初RPS,Unstable.年数,Unstable.增长率,Unstable.期末利润率,Unstable.期末EPS,Rapid.增长率,Rapid.期末EPS,内在价值,现价,安全边际%,估测PE
名称,,,,,,,,,,,
网易,32.89,0.00,0.0,21.0,6.91,16.6,32.08,92.04,134.00,-45.59,19.40
腾讯,64.88,0.00,0.0,33.4,21.67,20.0,134.17,384.94,347.20,9.80,16.02
快手,24.46,4.75,20.0,20.0,11.63,16.6,26.05,74.73,54.00,27.74,9.63
京东,334.96,1.75,30.0,2.5,13.25,16.6,47.05,134.99,152.90,-13.26,15.09
美团,39.76,1.75,30.0,20.0,12.59,20.0,56.64,162.50,130.30,19.81,14.24
拼多多,13.36,1.75,50.0,24.0,6.52,20.0,29.34,84.17,78.93,6.22,16.66
苹果,23.97,0.00,0.0,24.6,5.90,14.9,23.65,67.85,159.28,-134.77,27.01
微软,27.22,0.00,0.0,33.1,9.01,16.8,42.57,122.14,273.78,-124.15,30.39


data_202305:


,期初RPS,Unstable.年数,Unstable.增长率,Unstable.期末利润率,Unstable.期末EPS,Rapid.增长率,Rapid.期末EPS,内在价值,现价,安全边际%,估测PE
名称,,,,,,,,,,,
网易,31.11,0.0,0.0,24.6,7.65,16.6,35.55,101.99,139.50,-36.78,18.23
腾讯,65.41,0.0,0.0,33.2,21.72,16.6,100.87,289.39,316.20,-9.26,14.56
快手,25.02,4.5,20.0,20.0,11.37,16.6,26.45,75.89,54.50,28.19,9.57
京东,362.10,1.5,20.0,2.5,11.90,16.6,43.90,125.95,130.00,-3.22,13.75
美团,41.26,1.5,30.0,20.0,12.23,16.6,45.12,129.46,116.40,10.09,11.98
拼多多,15.54,1.5,50.0,24.0,6.85,16.6,25.28,72.52,71.42,1.52,13.12
苹果,24.02,0.0,0.0,24.5,5.88,14.9,23.60,67.71,175.43,-159.09,29.81
微软,27.75,0.0,0.0,33.3,9.24,16.8,43.67,125.27,332.89,-165.73,36.02


data_202309:


,期初RPS,Unstable.年数,Unstable.增长率,Unstable.期末利润率,Unstable.期末EPS,Rapid.增长率,Rapid.期末EPS,内在价值,现价,安全边际%,估测PE
名称,,,,,,,,,,,
网易,32.75,0.00,0.0,26.14,8.56,16.6,39.76,114.07,158.30,-38.77,18.49
腾讯,66.03,0.00,0.0,34.71,22.92,16.6,106.46,305.43,312.20,-2.22,13.62
快手,26.02,4.25,20.0,20.00,11.29,16.6,27.31,78.36,63.70,18.71,10.83
京东,362.80,1.25,20.0,2.50,11.39,16.6,43.67,125.29,121.00,3.42,12.87
美团,43.37,1.25,30.0,20.00,12.04,16.6,46.16,132.43,122.00,7.87,12.28
拼多多,16.15,1.25,50.0,25.08,6.72,16.6,25.77,73.94,99.81,-34.98,17.99
苹果,24.10,0.00,0.0,24.69,5.95,14.9,23.86,68.46,177.97,-159.96,29.91
微软,28.36,0.00,0.0,34.17,9.69,16.8,45.79,131.36,329.06,-150.50,33.96
英伟达,13.11,0.00,0.0,31.58,4.14,36.0,89.62,257.10,439.66,-71.01,106.20


data_202312:


,期初RPS,Unstable.年数,Unstable.增长率,Unstable.期末利润率,Unstable.期末EPS,Rapid.增长率,Rapid.期末EPS,内在价值,现价,安全边际%,估测PE
名称,,,,,,,,,,,
网易,34.81,0,0.0,26.34,9.17,16.6,42.59,122.20,140.60,-15.06,15.33
腾讯,69.60,0,0.0,32.72,22.77,16.6,105.77,303.44,293.60,3.24,12.89
快手,27.76,4,20.0,20.00,11.51,16.6,28.93,83.00,52.95,36.21,8.50
京东,374.15,1,20.0,2.50,11.22,16.6,44.72,128.28,112.50,12.30,11.69
美团,46.65,1,30.0,20.00,12.13,16.6,48.32,138.62,81.90,40.92,7.87
拼多多,19.08,1,50.0,27.52,7.88,16.6,31.37,90.00,146.31,-62.56,21.66
苹果,24.22,0,0.0,25.27,6.12,16.6,28.43,81.56,192.53,-136.07,31.46
微软,29.24,0,0.0,35.77,10.46,16.6,48.59,139.39,376.04,-169.77,35.95
英伟达,18.00,0,0.0,45.06,8.11,30.0,111.80,320.75,495.22,-54.39,61.06
